# The JuMP ecosystem for mathematical optimization

## JuliaCon 2018

## Juan Pablo Vielma
## MIT Sloan

## Minimum # of Passports to Visit all Countries?
[![Passport Index](img/passportindex.jpg "Passport Index")](https://www.passportindex.org)

199 passports = $10^{33}$ times the age of the universe to enumerate at $10^{17}$ flops!

In [ ]:
# Download data from https://github.com/ilyankou/passport-index-dataset
#;git clone https://github.com/ilyankou/passport-index-dataset.git
data = readdlm(joinpath("passport-index-dataset","passport-index-country-names.csv"),',')
cntr = data[2:end,1]
vf = (x ->  x == -1 || x == 3 ? 1 :0).(data[2:end,2:end]);

## (Constrained) Mathematical Optimization and JuMP

$$
\begin{align*}
\min_{x,y} &&\quad \sum_{\operatorname{cntr} \;\in\; \operatorname{World}} \operatorname{pass}_{\,\operatorname{cntr}} \\
\text{s.t.}&&\quad  \operatorname{vf}(\operatorname{cntr},\operatorname{dst}) \cdot \operatorname{pass}_{\,\operatorname{cntr}} &\geq 1\quad  &\quad& \forall \; \operatorname{dst} \;\in \; \operatorname{World}\\
 &&\operatorname{pass}_{\,\operatorname{cntr}}  &\in \{0,1\}&\quad& \forall \; \operatorname{cntr}\in \; \operatorname{World}.
\end{align*}
$$

In [ ]:
using JuMP, GLPK
model = Model(with_optimizer(GLPK.Optimizer))
@variable(model, pass[1:length(cntr)], Bin)
@constraint(model, [j=1:length(cntr)], sum( vf[i,j]*pass[i] for i in 1:length(cntr)) >= 1)
@objective(model, Min, sum(pass))
JuMP.optimize(model)
print(JuMP.objectivevalue(model)," passports: ",join(cntr[find(JuMP.resultvalue.(pass))],", "))

## JuMP: Started by Students Leading a Vibrant Community



 ![JuMP Community](img/jumpcommunity.png "JuMP Community")





# You Can Learn Optimization Using JuMP

![Nestor](img/nestor.png "Nestor")


### You Can Develop New Methods Using Julia / JuMP

* Mixed-Integer Nonlinear Solvers from MIT and LANL:
    - Pajarito.jl, Juniper.jl, POD.jl and Katana.jl
* Porting experimental Interior Point Method from Matlab in ~ week:
    - 45x speed with linear algebra options
    - Already good out-of-the-box performance:

    
| Instance        | Matlab           | Julia   | Instance        | Matlab           | Julia  | Instance        | Matlab           | Julia   | Instance        | Matlab           | Julia  |
| ----------------|-----------------:|--------:|----------------:|-----------------:|-------:|----------------|-----------------:|--------:|----------------:|-----------------:|-------:|
| dense lp        | 5.8              | 4.1     | lotka-volt      |0.47              |0.38 | butcher         | 0.63             |    0.41 | reac-diff       | 0.32             |    0.23 |
| envelope        | 0.085            |   0.043 | motzkin         | 0.35             |    0.24 | caprasse        | 1.38             |    1.87 | robinson        | 0.34             |    0.23 |


    

# JuMP is Composable and Uses the Julia Ecosystem

[Drone Control Demo](https://github.com/juan-pablo-vielma/Dagstuhl-Seminar-18081/blob/master/Polynomial.ipynb) prepared in ~1 week:


![drone](img/drone.png "drone")

Running on Julia 0.6 / JuMP 0.18 (0.7/0.19 soon).


# JuMP is Evolving

* JuMP 0.19: Towards JuMP 1.0
    * MathOptInterface.jl replaces MathProgBase.jl
* Pros: More "precise" control
    * Extensible constraint types 
    * Flexible "Model" attributes (e.g. warm-starts)
    * Clear status codes (cf. CPXMIP_OPTIMAL_INFEAS)
    * Efficient problem modifications
    * ...
* "Cons": 
    * Syntax changes ("no" changes from 0.19 to 1.0)
    * Some "features" dissapear "temporarily" (e.g. solver-independent callbacks, `norm(x)`)
        * Correct versions will come back or may be available through extensions
* Status:
    * [0.19 master is passing on Julia 0.7](https://github.com/JuliaOpt/JuMP.jl/pull/1399)
    - JuMP 0.18 in Julia 0.6 should be ok for the 2018-2019 academic year
    - **MY** (very conservative) deadline for 1.0-like stability/documentation/etc. = March, 2019



![NumFocus](img/NumFocus.png "NumFocus")
Pending "branding" transition:
![juliaopttojump](img/juliaopttojump.png "juliaopttojump") 




# Overview

1. Introduction to JuMP, noting syntax changes for 0.19
2. Introduction to MOI and associated concepts
3. Some discussion on efficiency and type stability (if time permits)

# More Information

* [Julia Opt site](http://www.juliaopt.org) (name change soon)
* [The Second Annual JuMP-dev Workshop Page](http://www.juliaopt.org/meetings/bordeaux2018/)
    * Slides and Videos of talks/tutorials
* [JuMP](https://github.com/JuliaOpt/JuMP.jl) and [MOI](https://github.com/JuliaOpt/MathOptInterface.jl) in github. 
* [JuMP-Dev channel in gitter](https://gitter.im/JuliaOpt/JuMP-dev)

    
**NOTE:** Almost everthing I know about JuMP "internals" I learned preparing this workshop!

# A step by step example
Let's see how we translate a simple, 2 variable LP to JuMP code.

$$
\begin{align*}
&\max_{x,y}& \quad x + 2y \\
&\text{s.t.}&\quad x + y &\leq 1 \\
&&0\leq x, y &\leq 1
\end{align*}
$$



Load JuMP, MathOptInterface (MOI), and GLPK (GNU LP/MIP solver):

In [ ]:
using JuMP  
using MathOptInterface # Replaces MathProgBase
# shortcuts
const MOI = MathOptInterface
const MOIU = MathOptInterface.Utilities

using GLPK # Loading the GLPK module for using its solver

Construct a model object (a container for variables, constraints, solver options, etc.):

In [ ]:
model = Model(with_optimizer(GLPK.Optimizer, msg_lev = 4));  
# Old syntax: model = Model(solver=GLPKSolverLP(msg_lev = 4)))

Define variables $0\leq x, y \leq 1$:

In [ ]:
@variable(model, 0 <= x <= 1)
@variable(model, 0 <= y <= 1);

Add constraint $x + y \leq 1$:

In [ ]:
@constraint(model, x + y <= 1)

Add objective $\max x + 2y$:

In [ ]:
@objective(model, Max, x + 2y)

To solve the optimization problem, call the `optimize` function.

In [ ]:
JuMP.optimize(model) # Old syntax: status = JuMP.solve(model)

We can then check the status of the optimization call.

In [ ]:
@show JuMP.hasresultvalues(model)
@show JuMP.terminationstatus(model) == MOI.Success
@show JuMP.primalstatus(model) == MOI.FeasiblePoint
@show JuMP.dualstatus(model) == MOI.FeasiblePoint

# New Solver Status

Much more details than old `:Optimal, :Unbounded, :Infeasible, :UserLimit, :Error, :NotSolved`

```julia 
@show JuMP.terminationstatus(model) == MOI.Success
```

In [ ]:
display(typeof(MOI.Success))

```julia
@show JuMP.primalstatus(model) == MOI.FeasiblePoint
```

In [ ]:
display(typeof(MOI.FeasiblePoint))

We can also inspect the solution values and optimal cost:

In [ ]:
@show JuMP.resultvalue(x)              # Old syntax: getvalue(x)
@show JuMP.resultvalue(y)              # Old syntax: getvalue(y)
@show JuMP.objectivevalue(model)       # Old syntax: getobjectivevalue(model)

I can also "name" constraints for later reference.

In [ ]:
model = Model(with_optimizer(GLPK.Optimizer, msg_lev = 0))
@variable(model, 0 <= x <= 1)
@variable(model, 0 <= y <= 1)
@constraint(model, inequality, x + y <= 1)     # <=============== constraint can be referenced later as "inequality"
@objective(model, Max, x + 2y)
JuMP.optimize(model)
@show JuMP.terminationstatus(model) == MOI.Success

constraint references  can by used to delete them

In [ ]:
JuMP.delete!(model, inequality)
JuMP.optimize(model)
@show JuMP.terminationstatus(model) == MOI.Success
@show JuMP.objectivevalue(model)  

Constraint references can be used to modify problem (see [MOI](MOI.ipynb#Model-modifications)) and to get duals (see [Topics notebook](Topics.ipynb#Duality)).

## Collections of variables/constraints and summation in JuMP

You can also create collections of variables like $x_i \geq 0 \quad \forall \; i\in\{1,\ldots,10\}$

In [ ]:
model = Model()
@variable(model, x[1:10] >= 0);

Also multidimensional indexing, separated by commas:

In [ ]:
@variable(model, y[1:10,["red","blue"]] <= 1);

and more complicated expressions like $\quad
i \leq z_{ij} \leq u_j \;\;\; \forall i \in \{1,...,10\}, j \in \{i+1, ..., 10\}
$:

In [ ]:
u = rand(10)
@variable(model, i <= z[i=1:10,j=(i+1):10] <= u[j]);

To specify conditions on the indexing, you can add conditionals inside the ``[...]`` block, separated by a semicolon:

In [ ]:
@variable(model, w[i=1:10, c=["red","blue"]; iseven(i) || c == "red"] >= 0)

Also easy to create constrainta like $ \sum _{i = 1}^{10} x_i \leq 1$:

In [ ]:
@constraint(model, sum(x[i] for i in 1:10) <= 1)

Or more complicated ones like 
$
\sum_{\substack{i\in\{1,...,10\}\\
                c\in\{"red","blue"\}}}
       coef(c) \cdot y_{ic} = 1
$

In [ ]:
coef = Dict("red" => 2, "blue" => 3)
@constraint(model, sum(coef[c]*y[i,c] for i in 1:10, c in ["red","blue"]) == 1)

or $
\sum_{i = 1}^{10} \sum_{j = i+1}^{10} 
       i \cdot j \cdot z_{ij} \leq
\sum_{\substack{i\in\{1,...,10\},
                c\in\{"red","blue"\} \\
                \text{s.t. } iseven(i) \text{ or } c = "red"}}
       i^2 \cdot w_{ic} 
$:

In [ ]:
@constraint(model, sum(i*j*z[i,j] for i in 1:10, j in (i+1):10) <=
               sum(i^2*w[i,c] for i in 1:10, c in ["red","blue"] if iseven(i) || c == "red"))

Can also do collections of constraints (named or unamed):
    $$ 
\begin{align}
x_i &\leq 0.9 \quad \forall i \in \{1,2,3\} \quad\text{ (large bounds)}\\
x_i &\leq 0.5 \quad \forall i \in \{4,5,6\} 
\end{align}
$$

In [ ]:
@constraint(model,largebounds[i=1:3], x[i] <= 0.9)
@constraint(model,[i=4:6], x[i] <= 0.5)

# New in JuMP 0.19: New Containers and Conventions

`JuMPDict` is replaced by `Base.Dict` and `JuMPArray` was rewritten (inspired by `AxisArrays`). Conventions apply for `@variable`, `@constraint`, `@expression`, `@NLconstraint`, `@NLexpression`, ...

In [ ]:
model = Model(with_optimizer(GLPK.Optimizer))
@variable(model, x[1:5, 1:5])            # Array
my_set = [:a, :b, :c]
@variable(model, w[1:5, my_set])        # JuMPArray
@variable(model, t[i = 1:5, 1:i])        # Dict
@variable(model, h[i = 1:5; isodd(i)]);  # Dict

Finally,  no more slicing error for JuMPArrays!

In [ ]:
model = Model(with_optimizer(GLPK.Optimizer))
set_1 = [:a, :b, :c]
set_2 = [:x, :y, :z]
@variable(model, x[set_1,set_2])
x[:,:z]

# A Warning on Performance and Type Stability 

`@variable` chooses the tightest applicable container while remaining **type stable**. 


In [ ]:
model = Model(with_optimizer(GLPK.Optimizer))
@variable(model, x[1:5, 1:5])            # Array
set_1 = Base.OneTo(5)
@variable(model, y[set_1, 1:5])          # Array
set_2 = 1:5
@variable(model, z[1:5, set_2])          # JuMPArray
set_3 = [:a, :b, :c]
@variable(model, w[set_2, set_3])        # JuMPArray
@variable(model, t[i=set_2, 1:i])        # Dict
@variable(model, h[i = 1:5; isodd(i)]);  # Dict

You can also request a container type (for more details see [Internals](Internals.ipynb#JuMP-Containers) notebook):

In [ ]:
model = Model(with_optimizer(GLPK.Optimizer))
@variable(model, x[1:5, 1:5], container = JuMPArray)
set_1 = 1:5
@variable(model, y[set_1, 1:5], container = Array)
set_2 = 2:3
# @variable(m, z[set_2, 1:5], container = Array)  # => Error instead of fallback to JuMPArray to preserve type stability

# Classes of Constraints Beyond Linear Inequalities

Broadcasted and two sided linear inequalities:

In [ ]:
A = [1.0 2.0; 3.0 4.0]
l = [4.0, 5.0]
u = [2.0, 3.0]
model = Model()
@variable(model, x[1:2])
@constraint(model, l .<= A*x .<= u)

## Quadratic Inequalities:

Both convex:

In [ ]:
model = Model()
@variable(model, x[1:2])
@constraint(model, x[1]^2 + x[2] <= 1)

and non-convex:

In [ ]:
@constraint(model, x[1]*x[2] - 1.0 == 0.0)

## Conic constraints including...

Semidefinite constraints:

In [ ]:
model = Model()                         # using CSDP; model = Model(with_optimizer(CSDP.CSDPOptimizer))
@variable(model, y[1:2,1:2], Symmetric)
@constraint(model, y in PSDCone())  
@variable(model, t)
@variable(model, w)
@SDconstraint(model,  [t 1; 1 -w] ⪰ [1 t; t -2])

Second order cone constraints:
$$ 
\begin{equation}
\left\| Ax+u \right\|_2 \leq t
\end{equation}
$$

In [ ]:
model = Model()
@variable(model, x[1:2])
@variable(model, t)
@constraint(model, [t;A*x+u] in SecondOrderCone())

Rotated second order cone constraints:
$$ 
\begin{equation}
\left\| Ax+u \right\|_2 \leq t \cdot w,\quad w\geq 0
\end{equation}
$$

In [ ]:
model = Model()
@variable(model, x[1:2])
@variable(model, t)
@variable(model, w)
@constraint(model, [t;w;A*x+u] in RotatedSecondOrderCone())

# Also "derivative based" nonlinear constraints

Remains unchanged:

In [ ]:
model = Model()
@variable(model, x)
@variable(model, y)

@NLobjective(model, Min, (1-x)^2 + 100(y-x^2)^2)
@NLconstraint(model, exp(x)+sin(x) <=0)